
# Location Web Scraping of DBLP Dataset

Jupyter Notebook for web scraping of the conferences locations of the DBLP dump.

For this process, the following CSV file is needed: ```out_dblp_papers.csv```. 
The above file must be generated running the ```preprocess_dblp.ipynb``` Notebook that is contained in the ```1 - Citation Dumps Preprocess``` folder of this Repository.

In particular, the following operations are going to be executed:
* Opening of the CSV peprocessed dump
* Obtaining the missing locations with queries to the DBLP website
* Fix of the locations format

Lastly, the entire preprocessed dump is going to be saved on disk in CSV format

In [1]:
# Libraries Import
import pandas as pd
import platform
import multiprocessing as mp 
import concurrent       
from location_scraper_multithread_utils import * 

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [2]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

### Multithreading Settings
Settings needed for the multithreaded queries to gather the missing conferences locations from the DBLP website.

Please specify the max number of workers below:

**Important Note**: during our tests we found out that DBLP refuses incoming connections if requests are made too frequently. You can read more about the DBLP Servers Rate Limit [here](https://dblp.org/faq/1474706.html).

We suggest to **set the number of workers to 1 if you have a large bandwidth** (over 100Mbps). Otherwise, you could try to set a higher value to make requests in parallel.

In [3]:
MAX_WORKERS = 1

You can also set a **sleep delay** (in seconds) between requests if that's not enough:

In [4]:
SLEEP_DELAY = 0.3 # Seconds

Special setting for the specific operating systems.

**Note**: Due to the latest MacOS releases' security measures, we need to use the spawn method instead of fork.

In [5]:
print(f"Notebook running on {platform.system()} OS: ")

if platform.system() == "Darwin" or platform.system() == "Windows": # MacOS and windows
    mp_ctx = mp.get_context("spawn")
    print("Spawn method has been set")
    
else: # other unix systems
    mp_ctx = mp.get_context("fork")
    print("Fork method has been set")

Notebook running on Darwin OS: 
Spawn method has been set


## Read of the CSV Preprocessed Dump

In [6]:
df_dblp_preprocessed = pd.read_csv(path_file_export + 'out_dblp_papers.csv', low_memory=False, index_col=[0])
df_dblp_preprocessed

,id,crossref,ee,key,url,year
1,3062808,conf/coopis/2000,https://doi.org/10.1007/10722620_29,conf/coopis/ChenD00,db/conf/coopis/coopis2000.html#ChenD00,2000
2,3062809,conf/coopis/2004-2,https://doi.org/10.1007/978-3-540-30469-2_46,conf/coopis/AbdellatifCL04,db/conf/coopis/coopis2004-2.html#AbdellatifCL04,2004
4,3062811,conf/coopis/2000,https://doi.org/10.1007/10722620_9,conf/coopis/PapastavrouCSP00,db/conf/coopis/coopis2000.html#PapastavrouCSP00,2000
5,3062812,conf/coopis/97,http://doi.ieeecomputersociety.org/10.1109/COO...,conf/coopis/BultzingsloewenKK97,db/conf/coopis/coopis97.html#BultzingsloewenKK97,1997
6,3062813,conf/coopis/2003,https://doi.org/10.1007/978-3-540-39964-3_50,conf/coopis/GiacolettoA03,db/conf/coopis/coopis2003.html#GiacolettoA03,2003
...,...,...,...,...,...,...
2990257,8958667,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_9,series/sapere/Besold13,db/series/sapere/sapere5.html#Besold13,2011
2990258,8958671,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_20,series/sapere/Steiner13,db/series/sapere/sapere5.html#Steiner13,2011
2990259,8958672,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_25,series/sapere/Armstrong13,db/series/sapere/sapere5.html#Armstrong13,2011
2990260,8958682,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_12,series/sapere/Freed13,db/series/sapere/sapere5.html#Freed13,2011


## Fix of the Conferences URL Format
The most of the papers have their URLs in the following formats:
* ```/db/conf/CONF_NAME/CONF_NAME+YEAR.html#PAPER_IDENTIFIER```
* ```/db/series/SERIES_NAME/SERIES_NAME+YEAR.html#PAPER_IDENTIFIER```

The final part of the URL that follows '.html' is totally useless for our purpose.

We must filter it, so we'll be able to filter the duplicated URLs to speed up our queries.

In [7]:
df_dblp_preprocessed.url = df_dblp_preprocessed.url.str.split('#').str[0]
df_dblp_preprocessed

,id,crossref,ee,key,url,year
1,3062808,conf/coopis/2000,https://doi.org/10.1007/10722620_29,conf/coopis/ChenD00,db/conf/coopis/coopis2000.html,2000
2,3062809,conf/coopis/2004-2,https://doi.org/10.1007/978-3-540-30469-2_46,conf/coopis/AbdellatifCL04,db/conf/coopis/coopis2004-2.html,2004
4,3062811,conf/coopis/2000,https://doi.org/10.1007/10722620_9,conf/coopis/PapastavrouCSP00,db/conf/coopis/coopis2000.html,2000
5,3062812,conf/coopis/97,http://doi.ieeecomputersociety.org/10.1109/COO...,conf/coopis/BultzingsloewenKK97,db/conf/coopis/coopis97.html,1997
6,3062813,conf/coopis/2003,https://doi.org/10.1007/978-3-540-39964-3_50,conf/coopis/GiacolettoA03,db/conf/coopis/coopis2003.html,2003
...,...,...,...,...,...,...
2990257,8958667,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_9,series/sapere/Besold13,db/series/sapere/sapere5.html,2011
2990258,8958671,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_20,series/sapere/Steiner13,db/series/sapere/sapere5.html,2011
2990259,8958672,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_25,series/sapere/Armstrong13,db/series/sapere/sapere5.html,2011
2990260,8958682,series/sapere/2013-5,https://doi.org/10.1007/978-3-642-31674-6_12,series/sapere/Freed13,db/series/sapere/sapere5.html,2011


## Obtaining the Conferences Locations from the DBLP Website
The conferences locations are going to be obtained from queries to the DBLP Website.

In [8]:
df_dblp_conferences = df_dblp_preprocessed[["url"]].copy()

Adding the new location column:

In [9]:
df_dblp_conferences['ConferenceLocation'] = np.nan
df_dblp_preprocessed['ConferenceLocation'] = np.nan

Drop of the duplicated conferences. We only need unique values.

In [10]:
df_dblp_conferences = df_dblp_conferences.drop_duplicates(subset="url")

print(f"Now we only need to search for the location of {df_dblp_conferences.__len__()} unique conferences")

Now we only need to search for the location of 40338 unique conferences


### Define of the Web Scraping Function
For simplicity, we're goint to use the same function used in the MAG web scraper. For this reason the web function is going to be passed as parameter.

In [11]:
def dblp_location_scraper(conferences_dataframe, mt_downloader_operation_function, dblp_url = "https://dblp.org/"):
    dict_conf_locations = {}      
    download_list = list(conferences_dataframe.url.values)

    executor = concurrent.futures.ProcessPoolExecutor(max_workers=int(MAX_WORKERS), mp_context=mp_ctx)
    futures = [executor.submit(mt_downloader_operation_function, conf_url, dblp_url, SLEEP_DELAY) for conf_url in download_list]

    for future in concurrent.futures.as_completed(futures):
        try:
            k, v = future.result()
        except Exception as e:
            print(f"{futures[future]} throws {e}")
        else:
            dict_conf_locations[k] = v
            pass

    # Converting the resulting dictionary to a dataframe
    df_conf_locations = pd.DataFrame(dict_conf_locations.items(), columns=['url', 'ConferenceLocation'])

    return df_conf_locations

#### Queries to the DBLP Website

Parallel execution of the queries to the DBLP website.

**Note**: this operation should take less than 12 hours, depending on your Internet speed.

In [12]:
df_conf_locations = dblp_location_scraper(df_dblp_conferences, mt_get_dblp_conf_location_from_dblp_operation, "https://dblp.org/")

https://dblp.org/db/conf/coopis/coopis2000.html
https://dblp.org/db/conf/coopis/coopis2004-2.html
https://dblp.org/db/conf/coopis/coopis97.html
https://dblp.org/db/conf/coopis/coopis2003.html
https://dblp.org/db/conf/coopis/coopis2004-1.html
https://dblp.org/db/conf/coopis/coopis2002.html
https://dblp.org/db/conf/coopis/coopis96.html
https://dblp.org/db/conf/coopis/coopis98.html
https://dblp.org/db/conf/coopis/coopis99.html
https://dblp.org/db/conf/coopis/coopis2001.html
https://dblp.org/db/conf/notere/notere2010.html
https://dblp.org/db/conf/notere/notere2011.html
https://dblp.org/db/conf/notere/notere2015.html
https://dblp.org/db/conf/notere/notere2008.html
https://dblp.org/db/conf/notere/notere2016.html
https://dblp.org/db/conf/goodtechs/goodtechs2020.html
https://dblp.org/db/conf/goodtechs/goodtechs2017.html
https://dblp.org/db/conf/goodtechs/goodtechs2019.html
https://dblp.org/db/conf/goodtechs/goodtechs2016.html
https://dblp.org/db/conf/goodtechs/goodtechs2018.html
https://dblp.o

Let's see how many conference locations have been fixed.

In [ ]:
df_conf_locations = df_conf_locations.dropna(subset = ['ConferenceLocation'])

print(f"Fixed {len(df_conf_locations.index)} over {len(df_dblp_conferences.index)} unique conferences")

Write of the fixed locations on disk:

In [ ]:
df_conf_locations.to_csv(path_file_export + 'out_dblp_locations_fixed.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_dblp_locations_fixed.csv')

## Join of the New Location Data with the Original Dataframe

In [ ]:
# Merge with the location dataframe
df_dblp_preprocessed = pd.merge(df_dblp_preprocessed, df_conf_locations, on=['url'], how='left')

# Combine the two columns
df_dblp_preprocessed['ConferenceLocation_x'] = df_dblp_preprocessed['ConferenceLocation_x'].fillna(df_mag_preprocessed['ConferenceLocation_y'])
df_dblp_preprocessed.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
df_dblp_preprocessed = df_dblp_preprocessed.drop(columns=['ConferenceLocation_y'])

df_dblp_preprocessed.iloc[:5]

Count of how many paper's conference locations are still missing

In [ ]:
n_missing = len(df_mag_preprocessed.index) - len(df_mag_preprocessed.dropna(subset = ['ConferenceLocation']).index)
print(f"{n_missing} missing paper's conference locations")

## Write of the Final CSV on Disk

Saving the resulting dataframe on disk in CSV format.

In [ ]:
# Write of the resulting CSV on Disk
df_dblp_preprocessed.to_csv(path_file_export + 'out_dblp_papers_and_locations.csv')

Check of the Exported CSV to be sure that everything went fine.

In [ ]:
# Check of the Exported CSV
df_dblp_exported_csv = pd.read_csv(path_file_export + 'out_dblp_papers_and_locations.csv', low_memory=False)
df_dblp_exported_csv